In [12]:
import pandas as pd
import scipy.constants
from scipy.optimize import curve_fit
from scipy.integrate import quad
from sklearn.metrics import r2_score
import seaborn
seaborn.set(style='whitegrid')
import numpy as np
import math
import matplotlib.pyplot as plt
import matplotlib.dates as md
import matplotlib.transforms as transforms
from matplotlib.ticker import LogFormatter
%matplotlib qt
# %matplotlib inline
import datetime as dt
pd.set_option('display.max_rows', 100) 

## Reading in the data and preparing DataFrames

#### MID

In [13]:
#read in the data
mid = pd.read_csv("MID.tsv", sep="\t", skiprows=lambda x: x <=11,  on_bad_lines="skip")
#print(mid.describe())
# #bring the time data into a new df:
time = pd.DataFrame(mid.iloc[:,0::2]) #- get every second column
masses = [2,4,12,15,16,18,28,32,40,44]

#add elapsed time columns
def elapsed_time(df):
    counter = len(df.columns)
    for idx, col in enumerate(df):
        timestamp = pd.to_datetime(df[col], format="%Y/%m/%d %H:%M:%S.%f")
        runtime = (timestamp - timestamp[0]).dt.total_seconds()
        colname = "Elapsed time, mass {}".format(masses[idx]) 
        df.insert(counter, colname, runtime)
        counter += 1
    return df
print(elapsed_time(time))
#print(time)
#print(elapsed_time)
time_sliced = time.iloc[:,10:].fillna(0) #select the time columns
print(time_sliced)

#convert elapsed times in time-dataframe back to a timestamp
def h2_time(df1,df2):
    startdate = pd.to_datetime(df1.iloc[2,0],format="%Y/%m/%d %H:%M:%S.%f")
    dic= {}
    for idx, mass in enumerate(masses):
        key = "Timestamp, mass {}".format(mass)
        content = df2.iloc[:, idx].dropna().values
        for row in content:
            dic.setdefault(key,[]).append((startdate+dt.timedelta(milliseconds=row*1000)))
    new_df = pd.DataFrame.from_dict(dic)
    return(new_df)

new_df = h2_time(time,time_sliced)
new_df.columns

               mass 2.0_time            mass 4.0_time  \
0    2023/05/05 14:14:38.980  2023/05/05 14:14:39.180   
1    2023/05/05 14:14:41.053  2023/05/05 14:14:41.253   
2    2023/05/05 14:14:46.127  2023/05/05 14:14:46.327   
3    2023/05/05 14:14:51.702  2023/05/05 14:14:51.902   
4    2023/05/05 14:14:57.277  2023/05/05 14:14:57.477   
..                       ...                      ...   
974  2023/05/05 15:45:05.027  2023/05/05 15:45:05.227   
975  2023/05/05 15:45:10.602  2023/05/05 15:45:10.802   
976  2023/05/05 15:45:16.177  2023/05/05 15:45:16.377   
977  2023/05/05 15:45:21.752  2023/05/05 15:45:21.952   
978  2023/05/05 15:45:27.327  2023/05/05 15:45:27.527   

              mass 12.0_time           mass 15.0_time  \
0    2023/05/05 14:14:39.380  2023/05/05 14:14:39.580   
1    2023/05/05 14:14:41.453  2023/05/05 14:14:41.653   
2    2023/05/05 14:14:46.527  2023/05/05 14:14:46.727   
3    2023/05/05 14:14:52.102  2023/05/05 14:14:52.302   
4    2023/05/05 14:14:57.677  

Index(['Timestamp, mass 2', 'Timestamp, mass 4', 'Timestamp, mass 12',
       'Timestamp, mass 15', 'Timestamp, mass 16', 'Timestamp, mass 18',
       'Timestamp, mass 28', 'Timestamp, mass 32', 'Timestamp, mass 40',
       'Timestamp, mass 44'],
      dtype='object')

#### Vaclogger

In [14]:
#reading vaclogger measurement file
vaclog=pd.read_csv("vaclog", sep="\t")
vaclog.head()

#adding an elapsed time column
vac_timestamps=pd.to_datetime(vaclog["Time"],format="%d/%m/%Y %H:%M:%S")
runtime=(vac_timestamps-vac_timestamps[0]).dt.total_seconds()    
vaclog.insert(2,"Elapsed time",runtime)
# vaclog.drop(index=vaclog.index[0], 
#         axis=0, 
#         inplace=True)
vaclog["Time"] = vac_timestamps.dt.strftime('%d-%m-%Y %H:%M:%S')
vaclog["Time"] = pd.to_datetime(vaclog["Time"],format='%d-%m-%Y %H:%M:%S')

vaclog.head()

,Live comments,Time,Elapsed time,100mbar inj,Barion_2,Barion_1,DUAL experiment,DUAL insulation,1mbar inj,helium,T-platinum,T-CERNOX,V_heater,DUAL inj line,I_emission,I_grid
0,NaN,2023-05-05 14:12:06,0.0,-0.025067,1.830000e-09,8.430000e-10,4.996000e-09,4.458000e-07,0.035404,329.0,-1.683,4268.558,0.000028,0.000008,NaN,NaN
1,NaN,2023-05-05 14:12:14,8.0,-0.025016,1.820000e-09,8.480000e-10,4.996000e-09,4.458000e-07,0.035395,329.0,-1.683,4268.782,0.000025,0.000008,NaN,NaN
2,NaN,2023-05-05 14:12:23,17.0,-0.021073,1.850000e-09,8.500000e-10,4.996000e-09,4.458000e-07,0.035408,329.0,-1.908,4268.558,0.000026,0.000008,NaN,NaN
3,NaN,2023-05-05 14:12:32,26.0,-0.020850,1.820000e-09,8.560000e-10,4.996000e-09,4.458000e-07,0.035400,329.0,-1.683,4268.782,0.000026,0.000008,NaN,NaN
4,NaN,2023-05-05 14:12:40,34.0,-0.020877,1.820000e-09,8.620000e-10,4.996000e-09,4.463000e-07,0.035375,329.0,-1.683,4269.343,0.000027,0.000008,NaN,NaN


#### HiVolta log

In [15]:
#Reading in the HiVolta measurement file 
hv = pd.read_csv("hv log", sep=",")

#hv.drop(columns=["Vmon1","Vmon2","Vmon3","Vmon4","Vmon5","Vmon6","Vmon7","Vmon8","Imon3","Imon4","Imon5","Imon6","Imon7","Imon8","Comments1"], inplace=True)
hv["hv_grid"]=[element * 1e-6 for element in hv["IMon1"]]
hv["I_em"]=[element * 1e-6 for element in hv["IMon2"]]

# hv.drop(hv.columns[hv.columns.str.contains('Unnamed',case = False)],axis = 1, inplace = True)
hv["I_em"]=abs(hv["I_em"])
hv["hv_grid"]=abs(hv["hv_grid"])

#adding a datetime column
datetime = pd.to_datetime(hv['Date'] + ' ' + hv['Time'],format="%d/%m/%Y %H:%M:%S.%f")
datetime = datetime.dt.strftime('%d-%m-%Y %H:%M:%S')
datetime = pd.to_datetime(datetime, format='%d-%m-%Y %H:%M:%S')
hv.insert(0,"datetime",datetime)
    
#making the column into a datetime object
hv_timestamps=pd.to_datetime(hv["Time"],format="%H:%M:%S.%f")
runtime=(hv_timestamps-hv_timestamps[0]).dt.total_seconds()    
hv.insert(2,"Elapsed time",runtime)

hv.head()



,datetime,Date,Elapsed time,Time,VMon1,VMon2,VMon3,VMon4,VMon5,VMon6,...,IMon3,IMon4,IMon5,IMon6,IMon7,IMon8,Comment,Unnamed: 19,hv_grid,I_em
0,2023-05-05 14:14:34,05/05/2023,0.000,14:14:34.800,0.0,0.06,0.04,0.08,0.0,0.0,...,0.0096,0.0067,0.0123,0.0231,0.0022,-0.0131,NaN,NaN,9.700000e-09,6.500000e-09
1,2023-05-05 14:14:35,05/05/2023,1.183,14:14:35.983,0.0,0.06,0.04,0.08,0.0,0.0,...,0.0096,0.0067,0.0123,0.0231,0.0022,-0.0131,NaN,NaN,9.700000e-09,6.500000e-09
2,2023-05-05 14:14:37,05/05/2023,2.366,14:14:37.166,0.0,0.06,0.04,0.08,0.0,0.0,...,0.0096,0.0067,0.0123,0.0231,0.0022,-0.0131,NaN,NaN,9.700000e-09,6.500000e-09
3,2023-05-05 14:14:38,05/05/2023,3.547,14:14:38.347,0.0,0.06,0.04,0.08,0.0,0.0,...,0.0096,0.0067,0.0123,0.0231,0.0022,-0.0131,NaN,NaN,9.700000e-09,6.500000e-09
4,2023-05-05 14:14:39,05/05/2023,4.728,14:14:39.528,0.0,0.06,0.04,0.08,0.0,0.0,...,0.0096,0.0067,0.0123,0.0231,0.0022,-0.0131,NaN,NaN,9.700000e-09,6.500000e-09


## Data processing

#### Comments

In [16]:
#print vaclog comments
print(pd.unique(vaclog["Live comments"]))

[nan 'LHe transfer complete' 'starting esd tests' 'esd tests complete']


In [17]:
#print hv comments
print(pd.unique(hv["Comment"]))
hv_comm = hv.loc[:,["datetime","Comment"]]
hv_comm.dropna(inplace=True)
hv_comm.index

[nan 'grid repelling' 'grid transparent' 'grid repelling 2'
 'grid transparent 2' 'grid repelling 3' 'grid transparent 3'
 'grid repelling 4' 'grid transparent 4' 'grid repelling 5'
 'grid transparent 5' 'grid repelling 6']


Int64Index([ 381,  382,  383,  384,  543,  544,  545,  546,  626,  627,  628,
             629,  866,  867,  868,  869, 2331, 2332, 2333, 2334, 2335, 2560,
            2561, 2562, 2563, 2801, 2802, 2803, 2804, 3317, 3318, 3319, 3320,
            3325, 3326, 3327, 3328, 3908, 3909, 3910, 3911, 3912, 3953, 3954,
            3955, 3956, 3957, 4165, 4166, 4167, 4168, 4169],
           dtype='int64')

#### CernOx temperature R-T conversion

In [18]:
#Temperature curve for CERNOX - for temp stability
A=[230.317302,-6170.1513,71837.9529,-477946.76,2.003668910085786e+6,-5.488690193047771e+6,9.830475663897528e+6,-1.111226817786569e+7,7.202477878914065e+6,-2.04194551328507e+6]

#specify fit parameters A, data (Resistance values)
def polyfit(params,data):
    total=[]
    for j in data: 
        exp=0
        for i in range(len(params)):
            exp += (params[i]/(math.log10(j))**i)
        total.append(10**exp)
    return(total)    


## Plotting

In [19]:
#define parameters for plotting
#annotation params
font = dict(size = "x-small", color ="green", style ="italic",rotation="vertical")


In [20]:
#Plotting the all the relevant MID spectrums
plt.figure(figsize=(14,8))

plt.plot(new_df["Timestamp, mass 2"],mid["mass 2.0_value"],marker=".", markersize=4,label='M2')
plt.plot(new_df["Timestamp, mass 4"],mid["mass 4.0_value"],marker=".", markersize=4,label='M4')
plt.plot(new_df["Timestamp, mass 12"],mid["mass 12.0_value"],marker=".", markersize=4,label='M12')
plt.plot(new_df["Timestamp, mass 15"],mid["mass 15.0_value"],marker=".", markersize=4,label='M16')
plt.plot(new_df["Timestamp, mass 16"],mid["mass 16.0_value"],marker=".", markersize=4,label='M15')
plt.plot(new_df["Timestamp, mass 18"],mid["mass 18.0_value"],marker=".", markersize=4,label='M18')
plt.plot(new_df["Timestamp, mass 28"],mid["mass 28.0_value"],marker=".", markersize=4,label='M28')
plt.plot(new_df["Timestamp, mass 32"],mid["mass 32.0_value"],marker=".", markersize=4,label='M32')
plt.plot(new_df["Timestamp, mass 40"],mid["mass 40.0_value"],marker=".", markersize=4,label='M40')
plt.plot(new_df["Timestamp, mass 44"],mid["mass 44.0_value"],marker=".", markersize=4,label='M44')

plt.xlabel('Timestamp')
plt.ylabel('Ion current (A)')
plt.legend(title="Mol masses", loc="upper right")
plt.title("S1 CU ESD sample out - MID scan")
plt.tick_params(axis="y", which='minor')
plt.grid(which='minor', axis='y')
plt.yscale('log')

ax = plt.gca()
ax.xaxis.set_major_formatter(md.DateFormatter('%H:%M'))
ax.xaxis.set_major_locator(plt.MaxNLocator(14))
for label in ax.get_xticklabels(which='major'):
    label.set(rotation=30, horizontalalignment='right')

#write comments
trans  = transforms.blended_transform_factory(
    ax.transData, ax.transAxes)
for i in hv_comm.index:
    plt.text(hv_comm["datetime"][i],0.95,str(hv_comm["Comment"][i]),fontdict=font, transform=trans)


plt.savefig(r'./graphs/S1 Cu ESD EGA out_MID.png')
plt.show()


In [21]:
#plotting HiVolta log
plt.figure(figsize=(12,6))

plt.plot(hv["datetime"],hv["hv_grid"],marker=".", markersize=4,label='Grid current')
plt.plot(hv["datetime"],hv["I_em"],marker=".", markersize=4,label='Emission current')
plt.xlabel('Timestamp')
plt.ylabel('Current (A)')
plt.legend(loc="upper right")
plt.title("S1 Cu ESD sample out_HiVolta log")
plt.tick_params(axis="y", which='minor')
plt.grid(which='minor', axis='y')
plt.yscale('log')

ax = plt.gca()
ax.xaxis.set_major_formatter(md.DateFormatter('%H:%M'))
ax.xaxis.set_major_locator(plt.MaxNLocator(10))
for label in ax.get_xticklabels(which='major'):
    label.set(rotation=30, horizontalalignment='right')
#comments
trans  = transforms.blended_transform_factory(
    ax.transData, ax.transAxes)
for i in hv_comm.index:
    plt.text(hv_comm["datetime"][i],0.8,str(hv_comm["Comment"][i]),fontdict=font, transform=trans)
    
plt.savefig(r'./graphs/S1 Cu ESD EGA out_HiVolta.png')
plt.show()


In [22]:
#Plotting BA2, BA1 pressure
plt.figure(figsize=(12,6))
plt.plot(vaclog["Time"],vaclog["Barion_2"],marker=".", markersize=4,label='Barion 2')
plt.plot(vaclog["Time"],vaclog["Barion_1"],marker=".", markersize=4,label='Barion 1')
plt.plot(vaclog["Time"],vaclog["DUAL experiment"],marker=".", markersize=4,label='DUAL experiment')

plt.xlabel('Timestamp')
plt.ylabel('Pressure (mbar)')
plt.legend()
plt.title("S1 CU ESD sample out - Total pressure evolution")
plt.tick_params(axis="y", which='minor')
plt.grid(which='minor', axis='y')
plt.yscale('log')

ax = plt.gca()
ax.xaxis.set_major_formatter(md.DateFormatter('%H:%M'))
ax.xaxis.set_major_locator(plt.MaxNLocator(10))
for label in ax.get_xticklabels(which='major'):
    label.set(rotation=30, horizontalalignment='right')    
       
plt.savefig(r'./graphs/S1 Cu ESD EGA out_Pressures.png')
plt.show()

In [23]:
## Plotting the temp evolution
plt.figure(figsize=(10,6))
plt.plot((vaclog["Time"]),polyfit(A,vaclog["T-CERNOX"]),marker=".", markersize=5,label='T-CERNOX')          
plt.xlabel('Timestamp')
plt.ylabel('Temperature (K)')
plt.legend()
plt.title("S1 CU ESD sample out - Temperature evolution")
plt.yscale('linear')
plt.tick_params(axis="y", which='minor')
plt.grid(which='minor', axis='y')

ax = plt.gca()
ax.xaxis.set_major_formatter(md.DateFormatter('%H:%M'))
ax.xaxis.set_major_locator(plt.MaxNLocator(10))
for label in ax.get_xticklabels(which='major'):
    label.set(rotation=30, horizontalalignment='right')
    
plt.savefig(r'./graphs/S1 Cu ESD EGA out_Temp.png')
plt.show()